# Calculating the SLP gradient in the Pacific (i.e., Walker circulation) in the LongRunMIP
#### Christopher Callahan
#### Christopher.W.Callahan.GR@dartmouth.edu

Raw Nino4 index minus raw Nino3 index.

#### Mechanics
Import dependencies

In [ ]:
import xarray as xr
import numpy as np
import sys
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
from matplotlib import rcParams
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy.io import loadmat
from matplotlib.patches import Polygon
from scipy import signal
from scipy import stats

Model names

In [ ]:
modelnames_file = ['CCSM3_abrupt2x','CCSM3_abrupt4x','CCSM3_abrupt8x', \
    'CESM104_abrupt2x','CESM104_abrupt4x','CESM104_abrupt8x', \
    'CNRMCM61_abrupt4x','GFDLCM3_1pct2x','GFDLESM2M_1pct2x','GISSE2R_1pct4x', \
    'GISSE2R_abrupt4x','HadCM3L_abrupt2x','HadCM3L_abrupt4x', \
    'HadCM3L_abrupt6x','HadCM3L_abrupt8x','IPSLCM5A_abrupt4x', \
    'MIROC32_1pct2x','MIROC32_1pct4x','MPIESM12_abrupt2x', \
    'MPIESM12_abrupt4x','MPIESM12_abrupt8x']

Data locations

In [ ]:
loc_psl = "~/" # change if using raw data
loc_out = "../Data/SLPG/"

Latitude and longitude information

In [ ]:
lat_min = -5
lat_max = 5

lon_min_w = 80
lon_max_w = 160
lon_min_e = 200
lon_max_e = 280

#### Analysis

Read in psl for each experiment, calculate gradient, output

In [ ]:
for i in np.arange(0,len(modelnames_file),1):

    model, exp = modelnames_file[i].split("_")
    print(modelnames_file[i])
    
    fname_exp_start = "psl_mon_"+modelnames_file[i]
    fname_control_start = "psl_mon_"+model+"_control"
    
    fname_exp = [f for f in os.listdir(loc_psl) if f.startswith(fname_exp_start)][0]
    fname_control = [f for f in os.listdir(loc_psl) if f.startswith(fname_control_start)][0]
    
    n1, n2, n3, n4, year_exp_nc = fname_exp.split("_")
    n1, n2, n3, n4, year_control_nc = fname_control.split("_")
    year_exp, nc = year_exp_nc.split(".")
    year_control, nc = year_control_nc.split(".")
    
    if ((model == "MIROC32") | (model == "MPIESM12")):
        psl_exp_global = xr.DataArray(xr.open_dataset(loc_psl+fname_exp,decode_times=False).data_vars["psl"]) 
        psl_control_global = xr.DataArray(xr.open_dataset(loc_psl+fname_control,decode_times=False).data_vars["psl"])
    else:
        psl_exp_global = xr.DataArray(xr.open_dataset(loc_psl+fname_exp).data_vars["psl"])
        psl_control_global = xr.DataArray(xr.open_dataset(loc_psl+fname_control).data_vars["psl"])
    
    
    if model == "HadCM3L":
        lat_psl = psl_exp_global.coords["latitude_1"]
        lon_psl = psl_exp_global.coords["longitude_1"]
    else:
        lat_psl = psl_exp_global.coords["lat"]
        lon_psl = psl_exp_global.coords["lon"]
            
    if lat_psl[0] > lat_psl[len(lat_psl.values)-1]:
        psl_west_exp = psl_exp_global.loc[:,lat_max:lat_min,lon_min_w:lon_max_w].mean(axis=(1,2))
        psl_east_exp = psl_exp_global.loc[:,lat_max:lat_min,lon_min_e:lon_max_e].mean(axis=(1,2))
        psl_west_control = psl_control_global.loc[:,lat_max:lat_min,lon_min_w:lon_max_w].mean(axis=(1,2))
        psl_east_control = psl_control_global.loc[:,lat_max:lat_min,lon_min_e:lon_max_e].mean(axis=(1,2))
    else:
        psl_west_exp = psl_exp_global.loc[:,lat_min:lat_max,lon_min_w:lon_max_w].mean(axis=(1,2))
        psl_east_exp = psl_exp_global.loc[:,lat_min:lat_max,lon_min_e:lon_max_e].mean(axis=(1,2))
        psl_west_control = psl_control_global.loc[:,lat_min:lat_max,lon_min_w:lon_max_w].mean(axis=(1,2))
        psl_east_control = psl_control_global.loc[:,lat_min:lat_max,lon_min_e:lon_max_e].mean(axis=(1,2))
    
    slpg_exp = psl_east_exp - psl_west_exp
    slpg_control = psl_east_control - psl_west_control
    
    
    # output
    
    slpg_exp.name = "slpg"
    slpg_exp.attrs["creation_date"] = str(datetime.datetime.now())
    slpg_exp.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
    slpg_exp.attrs["data_description"] = "SLP gradient (east-west) from  "+model+" "+exp
    slpg_exp.attrs["created_from"] = "Calculate_SLPGradient.ipynb"
        
    fname_out_slpg_exp = loc_out+"slpg_"+model+"_"+exp+".nc"
    slpg_exp.to_netcdf(fname_out_slpg_exp,mode="w")
    print(fname_out_slpg_exp)
    
    
    slpg_control.name = "slpg"
    slpg_control.attrs["creation_date"] = str(datetime.datetime.now())
    slpg_control.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
    slpg_control.attrs["data_description"] = "SLP gradient (east-west) from "+model+" control"
    slpg_control.attrs["created_from"] = "Calculate_SLPGradient.ipynb"
        
    fname_out_slpg_control = loc_out+"slpg_"+model+"_control.nc"
    slpg_control.to_netcdf(fname_out_slpg_control,mode="w")
    print(fname_out_slpg_control)